# Optimization

of linear-quadratic problems.

Many optimization problems in finance and econometrics involve linear-quadratic objectives/constraints. This notebook illustrates how the package [Convex.jl](https://github.com/jump-dev/Convex.jl) can be used for this. The example is (for pedagogical reasons) the same as in the other notebooks on optimization. Otherwise, the methods illustrated here are well suited for cases when the objective involves the portfolio variance ($ w'\Sigma w $) or when the estimation problem is based on minimizing the sum of squared residuals ($u'u$).

The notebook also uses [SCS.jl](https://github.com/jump-dev/SCS.jl) (for the optimization algorithm).

## Load Packages and Utility Functions

In [1]:
using Printf, LinearAlgebra, Convex, SCS

include("jlFiles/printmat.jl")

printyellow (generic function with 1 method)

## A Linear-Quadratic Minimization Problem

with/without constraints.

We specify a matrix $Q$ and a vector $c$ and write the loss function as $b'Qb + c'b$ where $b$ are the choice variables. 

We consider several cases below: no restrictions on $b$, bounds on $b$, a linear equality restriction and a non-linear inequality restriction.

In [2]:
Q = [1  0;           #we want to minimize b'Q*b + c'b
     0  16]          #this is the same as minimizing (x-2)^2 + (4y+3)^2 
c = [-4, 24]

2-element Vector{Int64}:
 -4
 24

## Unconstrained Minimization

In [3]:
n  = length(c)
b  = Variable(n)              #define the choice variables
L1 = quadform(b,Q)            #part 1 of the objective, b'Q*b
L2 = dot(c,b)                 #part 2, c'b

problem = minimize(L1+L2)
solve!(problem,()->SCS.Optimizer(verbose=false))
b_sol = problem.status == Convex.MOI.OPTIMAL ? evaluate(b) : NaN

println("Unconstrained minimization: the solution should be (2,-3/4)")
printmat(b_sol)

Unconstrained minimization: the solution should be (2,-3/4)
     2.000
    -0.750



## Constrained Minimization

In [4]:
c1 = [2.75 <= b[1],b[2] <= -0.3]     #bounds on the solution

problem = minimize(L1+L2,c1)
solve!(problem,()->SCS.Optimizer(verbose=false))
b_sol = problem.status == Convex.MOI.OPTIMAL ? evaluate(b) : NaN

println("with bounds on the solution: the solution should be (2.75,-0.75)")
printmat(b_sol)

with bounds on the solution: the solution should be (2.75,-0.75)
     2.750
    -0.750



In [5]:
c2 = dot([1,2],b) == 3     #equality constraint

problem = minimize(L1+L2,c2)
solve!(problem,()->SCS.Optimizer(verbose=false))
b_sol = problem.status == Convex.MOI.OPTIMAL ? evaluate(b) : NaN

println("equality constraint: the solution should be (4,-1/2)")
printmat(b_sol)

equality constraint: the solution should be (4,-1/2)
     4.000
    -0.500



In [6]:
c3 = b[2] + square(b[1]-4) <= 0      #non-linear inequality constraint

problem = minimize(L1+L2,c3)
solve!(problem,()->SCS.Optimizer(verbose=false))
b_sol = problem.status == Convex.MOI.OPTIMAL ? evaluate(b) : NaN

println("non-linear inequality constraint: the solution should be close to (3.1,-0.79)")
printmat(b_sol)

non-linear inequality constraint: the solution should be close to (3.1,-0.79)
     3.112
    -0.789

